In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [5]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b k_B}`
        
   #Args:
   #    a: Term related with the attraction between particles in
   #    J m^3/ mol^2.\n
   #    b: Term related with the volume that is occupied by one 
   #    mole of the molecules in 10^-3 m^3/mol.\n
   #    
   #Returns:
   #    p_c: Critical pressure in Pa.\n
   #    v_c: Critical volume in m^3/mol.\n
   #    T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [6]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1

    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [57]:
#(a, b, element's name)
parameters = [(5.536, 0.03049, 'Water'),
             (4.225, 0.0371, 'Ammonia'),
             (1.358, 0.02789, 'Nitric oxide'),
             (4.25, 0.05105, 'Xenon')]

#I want to show the same range in v so you can compare the isotherms of all the elements
#so, let's calculate the critic point of the first one and use as a reference for the rest

p_c1, v_c1, T_c1 = calculate_critic(parameters[0][0], parameters[0][1])
v_values = np.linspace(0.8*parameters[0][1], 10*v_c1, 500)

scale_x = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y = bqs.LinearScale(min = 0.0, max = 2.0*p_c1)

axis_x = bqa.Axis(scale=scale_x,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

main_block = widgets.VBox([])
h_block_1 = widgets.HBox([])

main_block.children = [h_block_1]

if len(parameters) > 3:

    h_block_2 = widgets.HBox([])
    main_block.children = [h_block_1, h_block_2]

for i in range(len(parameters)):
    
    elem = parameters[i]
    
    a = elem[0]
    b = elem[1]
    name = elem[2]
    
    p_c, v_c, T_c = calculate_critic(a, b)
    
    T_values = [0.6*T_c, 0.8*T_c, T_c, 1.2*T_c]
    
    isotherms = get_absolute_isotherms(a, b, v_values, T_values)
    
    block = widgets.VBox([], layout=widgets.Layout(width='33%'))
    
    marks = []

    for j in range(len(isotherms)):

        marks.append(bqm.Lines(
                    x = v_values, 
                    y = isotherms[j], 
                    scales = {'x': scale_x, 'y': scale_y}, 
                    opacities = [1.0],
                    visible = True, #True, #t == '1.00',
                    colors = ['red'],
                    labels = [str(T_values[i])],
                    ))
    
    block.children = [
                Figure(title=name,
                marks=marks,
                axes=[axis_x, axis_y],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(width='90%', height='300px')
                      ),
                #widgets.HTML(value='<p>'+str(name)+'</p>', layout=widgets.Layout(align_self='center')),
                widgets.HTML(value='<p>a: '+str(a)+'</p>', layout=widgets.Layout(align_self='center')),
                widgets.HTML(value='<p>b: '+str(b)+'</p>', layout=widgets.Layout(align_self='center'))
    ]  

    if i > 2:
        h_block_2.children = h_block_2.children + (block,)
        
    else:
        h_block_1.children = h_block_1.children + (block,)
        
main_block